In [1]:
%matplotlib inline
import sys
import os
import pathlib
import math
import logging
import psycopg2
from psycopg2 import extras
import numpy as np
import pandas
from matplotlib import pyplot

In [2]:
_logger = logging.getLogger( __name__ )
_logger.propagate = False
_logout = logging.StreamHandler( sys.stderr )
_logger.addHandler( _logout )
_logout.setFormatter( logging.Formatter( f'[%(asctime)s - %(levelname)s] - %(message)s' ) )
_logger.setLevel( logging.INFO )

In [3]:
with open( pathlib.Path( os.getenv("HOME") ) / "secrets/decatdb_desi_desi" ) as ifp:
    dbuser, dbpasswd = ifp.readline().strip().split()
db = psycopg2.connect( dbname='desidb', host='decatdb.lbl.gov', user=dbuser, password=dbpasswd,
                      cursor_factory=psycopg2.extras.RealDictCursor )

In [4]:
targs = {}
for table in [ "maintargets", "backuptargets", "secondarytargets", "sv1targets", "sv2targets", "sv3targets" ]:
    _logger.info( f"Matching to {table}" )
    extracolumns = ",t.catalog" if table=="secondarytargets" else ",t.bgs_target,t.mws_target,t.survey"
    db.rollback()
    cursor = db.cursor()
    q = ( f"SELECT mh.snname, mh.index, mh.sn_z, t.targetid, t.desi_target {extracolumns} "
          f"INTO TEMP TABLE targetmatch FROM static.mosthosts mh "
          f"LEFT JOIN general.{table} t ON q3c_join( mh.ra, mh.dec, t.ra, t.dec, 0.5/3600. )" )
    cursor.execute( q )
    extracolumns = ",catalog" if table=="secondarytargets" else ",COALESCE(bgs_target,-1) AS bgs_target,COALESCE(mws_target,-1) AS mws_target,survey"
    cursor.execute( f"SELECT snname,index,sn_z,COALESCE(targetid,-1) AS targetid,COALESCE(desi_target,-1) AS desi_target "
                    f"{extracolumns} FROM targetmatch" )
    targs[table] = pandas.DataFrame( cursor.fetchall() )

_logger.info( "Done." )
cursor.close()
db.rollback()

[2024-02-05 11:32:50,600 - INFO] - Matching to maintargets
[2024-02-05 11:36:59,154 - INFO] - Matching to backuptargets
[2024-02-05 11:36:59,425 - INFO] - Matching to secondarytargets
[2024-02-05 11:36:59,644 - INFO] - Matching to sv1targets
[2024-02-05 11:36:59,951 - INFO] - Matching to sv2targets
[2024-02-05 11:37:00,231 - INFO] - Matching to sv3targets
[2024-02-05 11:40:59,888 - INFO] - Done.


In [5]:
targs['maintargets']

,snname,index,sn_z,targetid,desi_target,bgs_target,mws_target,survey
0,13ut,1,0.004563,-1,-1,-1,-1,None
1,AGN_2000,1,0.242000,-1,-1,-1,-1,None
2,ZTF19acmbhvv,1,0.062406,39633303501801591,5764607523034234880,514,0,main
3,ZTF19aclljyq,1,0.079000,39627959992058032,5764607523034234880,131074,0,main
4,ZTF19acllajw,1,0.042000,39627663630927025,5764607523034234880,131074,0,main
...,...,...,...,...,...,...,...,...
20620,ZTF19acujyzv,1,-9999.000000,39633472859408507,5764607523034234880,514,0,main
20621,ZTF19acudmhc,1,-9999.000000,-1,-1,-1,-1,None
20622,ZTF19acudlxq,1,-9999.000000,39633504077614045,5764607523034234880,514,0,main
20623,ZTF19actairt,3,-9999.000000,39633329804283378,4611686018427390562,0,0,main


In [6]:
( targs['maintargets']['targetid'] < 0 ).sum()

8807

In [7]:
( targs['sv3targets']['targetid'] < 0 ).sum()

8747

In [8]:
( targs['secondarytargets']['targetid'] <0 ).sum()

20174

In [9]:
targs['secondarytargets']['catalog'].isna().sum()

20174

In [10]:
targs['secondarytargets']

,snname,index,sn_z,targetid,desi_target,catalog
0,13ut,1,0.004563,-1,-1,None
1,AGN_2000,1,0.242000,-1,-1,None
2,ZTF19acmbhvv,1,0.062406,-1,-1,None
3,ZTF19aclljyq,1,0.079000,-1,-1,None
4,ZTF19acllajw,1,0.042000,-1,-1,None
...,...,...,...,...,...,...
20169,ZTF19acujyzv,1,-9999.000000,-1,-1,None
20170,ZTF19acudmhc,1,-9999.000000,-1,-1,None
20171,ZTF19acudlxq,1,-9999.000000,-1,-1,None
20172,ZTF19actairt,3,-9999.000000,-1,-1,None
